In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def get_image_data(path, resize=None, dtype=np.float32):
    filepath = glob.glob(path)[0]
    image = cv2.imread(filepath).astype(dtype)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ratio = None
    if resize!=None:
        if isinstance(resize, (list, tuple)):
            resize = (resize[1], resize[0])
        else:
          assert type(resize)==int
          height, width, channels = image.shape
          w = width if width<height else height
          ratio = resize/w
          resize = (int(width*ratio), int(height*ratio))
        image = cv2.resize(image, resize, interpolation=cv2.INTER_CUBIC)
    return image, ratio
  
def preprocessing(image):
    image[:, :, 0] -= 123.68
    image[:, :, 1] -= 116.779
    image[:, :, 2] -= 103.939  
    return image
  
  
def deprocessing(image):
    image[:, :, 0] += 123.68
    image[:, :, 1] += 116.779
    image[:, :, 2] += 103.939   
    image = np.clip(image, 0, 255).astype('uint8')   
    return image
  

def Gram_Matrix(x):
    assert len(x.shape) == 3, "wrong shape of input tensor, should be: (height, width, n_channel)"   
    # flatten the imgae tensor
    x = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # compute the gram matrix, shape : (n_channel, n_channel)
    gram = K.dot(x, K.transpose(x))

    return gram

def content_loss_fcn(content, generated):
    return 1/2 * tf.reduce_sum(tf.pow((generated - content), 2))

def style_loss_fcn(style, generated, style_weight):
    gram_style = Gram_Matrix(style)
    gram_generated = Gram_Matrix(generated)                   # Gram Matrix of Generated feature vector
    
    height, width, n_channel = generated.shape
    size = height.value * width.value
    norm_factor = 1 / (4 * (n_channel.value ** 2) * (size ** 2))
    
    return style_weight * norm_factor * tf.reduce_sum(tf.pow((gram_generated - gram_style), 2))

  
def total_variation_loss(x):
    assert len(x.shape)==4, "wrong shape for input of total variation loss, need to fix"
    _, height, width, _ = x.shape
    
    height = height.value
    width  = width.value
  
    a = tf.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = tf.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    
    return tf.reduce_sum(tf.pow(a + b, 1.25))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if __name__ == '__main__':

    # parameters for general model settings:
    content_img_path    = '/content/drive/My Drive/Colab Notebooks/Image Style Transformation/target_img/gate.jpg'
    style_img_path      = '/content/drive/My Drive/Colab Notebooks/Image Style Transformation/style_img/style5.jpg'
    image_resize        = 512
    rescale_image       = False
    content_blocks      = ['block4_conv2']
    style_blocks        = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    style_weights       = {block : 1/len(style_blocks) for block in style_blocks}
    loss_ratio          = 1e-3
    tv_weight           = 5e-8
    model_type          = 0         # 0 == vgg16, 1 == vgg19
    optimizer_type      = 0         # Options are 0 == Adam Optimizer, 1 == L-BFGS-B Optimizer.

    # hyper parameter for Adam Optimizer:
    learning_rate       = 1e+1
    beta_1              = 0.8
    beta_2              = 0.999
    epsilon             = 1e-08
    iteration           = 150

    # Load content and style images, initialize generated image with random noise
    content_image, rescale = get_image_data(content_img_path, image_resize)
    style_image, _         = get_image_data(style_img_path, content_image.shape[:2])
    print("Image Shape: ", content_image.shape)
    
    generated_image = tf.random.normal(shape=content_image.shape, stddev=np.std(content_image))
    generated_image = tf.Variable(generated_image, dtype=tf.float32, name='random_noise', trainable=True)


    # images preprocessing
    content_image = preprocessing(content_image)
    style_image   = preprocessing(style_image)

    image_shape = (1,) + content_image.shape  # shape = (1, height, width, 3)
    
    content_image   = content_image.reshape(image_shape)
    style_image     = style_image.reshape(image_shape)
    init_tensor    = tf.reshape(generated_image, shape=image_shape)


    # Load pretrained model
    with tf.compat.v1.variable_scope('pretrained_model'):
      if model_type == 0:
        model = VGG16(weights='imagenet', input_tensor=init_tensor, include_top=False)
      elif model_type == 1:
        model = VGG19(weights='imagenet', input_tensor=init_tensor, include_top=False)
        
      keras_variables = [var.name for var in tf.global_variables() if 'pretrained_model' in var.name]
    
    output_dict = {layer.name: layer.output for layer in model.layers}
        

    # Session
    sess = K.get_session() 
    K.set_session(sess)

    content_features = {}
    style_features = {}
    
    for block in content_blocks:
      feature_maps = sess.run(output_dict[block], feed_dict={init_tensor: content_image})[0]
      content_features[block] = tf.constant(feature_maps, dtype=tf.float32)

    for block in style_blocks:
      feature_maps = sess.run(output_dict[block], feed_dict={init_tensor: style_image})[0]
      style_features[block] = tf.constant(feature_maps, dtype=tf.float32)#Gram_Matrix(feature_maps)
    
    # Content Loss
    content_loss = 0
    for block in content_blocks:
      generated = output_dict[block][0] 
      content = content_features[block]
      content_loss +=  content_loss_fcn(content, generated)
    
    # Style Loss 
    style_loss = 0
    for block in style_blocks:
      generated = output_dict[block][0]            
      style = style_features[block]                        # Gram Matrix of Style feature vector
      w = style_weights[block]
      style_loss += style_loss_fcn(style, generated, w)
      
    # Total Variation Loss
    tv_loss = tv_weight * total_variation_loss(init_tensor)
      
    loss = loss_ratio * content_loss + style_loss + tv_loss


    # Minimize cost with optimizers
    trainble_variables = [var for var in tf.global_variables() if 'pretrained_model' not in var.name]  # Should not train the weights of pretrained model.
    if optimizer_type == 0:
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta_1, beta2=beta_2, epsilon=epsilon).minimize(loss, var_list=trainble_variables)
    elif optimizer_type == 1:
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(loss, var_list=trainble_variables, method='L-BFGS-B', options={'maxiter': iteration})


    # Initialize
    uninitialize_variables = [var for var in tf.global_variables() if var.name not in keras_variables]
    sess.run(tf.variables_initializer(uninitialize_variables))

    #show_weights_histogram(model)
     
    # Training
    with sess.as_default():
      if optimizer_type == 0:  # Adam Optimizer
        for i in range(iteration):  
            _cost, _c_cost, _s_cost, _tv_cost, _ = sess.run([loss, content_loss, style_loss, tv_loss, optimizer])

            if i % ((iteration // 10)) == 0:
                print('iter : {}'.format(i + 1), 'total loss : {:.2f}'.format(_cost),
                      'content_loss : {:.2f}'.format(_c_cost), 'style_loss : {:.2f}'.format(_s_cost))
      if optimizer_type == 1:  # L-BFGS-B Optimizer 
        _iter = 0
        def callback(_cost, _c_cost, _s_cost, _tv_loss):
            global _iter
            if _iter % ((iteration // 10)) == 0:
              print('iter : {}'.format(_iter + 1), 'total loss : {:.2f}'.format(_cost),
                    'content_loss : {:.2f}'.format(_c_cost), 'style_loss : {:.2f}'.format(_s_cost),
                     'tv_loss : {:.2f}'.format(_tv_loss))
            _iter += 1

        optimizer.minimize(sess, fetches=[loss, content_loss, style_loss, tv_loss], loss_callback=callback)
        
      
    print("training complete.")
    
    generated_image = sess.run(init_tensor)[0]
    generated_image = deprocessing(generated_image)
    
    if rescale_image == True:
      generated_image = cv2.resize(generated_image, None, fx=1/rescale, fy=1/rescale, interpolation=cv2.INTER_CUBIC)
    
    print("Final Image Shape =", generated_image.shape)
    
    K.clear_session()

Image Shape:  (512, 682, 3)
iter : 1 total loss : 9112537088.00 content_loss : 86548234240.00 style_loss : 9025968128.00
iter : 16 total loss : 118495224.00 content_loss : 54845386752.00 style_loss : 63632376.00
iter : 31 total loss : 75959584.00 content_loss : 44230250496.00 style_loss : 31711924.00
iter : 46 total loss : 59497320.00 content_loss : 35081568256.00 style_loss : 24398340.00
iter : 61 total loss : 50466924.00 content_loss : 29716180992.00 style_loss : 20733308.00
iter : 76 total loss : 44658384.00 content_loss : 26252421120.00 style_loss : 18388498.00
iter : 91 total loss : 40513892.00 content_loss : 23792375808.00 style_loss : 16704020.00
iter : 106 total loss : 37378020.00 content_loss : 21965117440.00 style_loss : 15395375.00
iter : 121 total loss : 34873672.00 content_loss : 20508205056.00 style_loss : 14347908.00
iter : 136 total loss : 32795304.00 content_loss : 19296960512.00 style_loss : 13480750.00
training complete.
Final Image Shape = (512, 682, 3)


In [ ]:
# Save image.

save_name = content_image_name[:-4].replace('-','_') + '_' + style_image_name[:-4].replace('-','_')
print('Final Image name =', save_name)

cv2.imwrite('{}.jpg'.format(save_name), cv2.cvtColor(generated_image, cv2.COLOR_RGB2BGR))

Final Image name = content_image_style_image


True

In [ ]:
from google.colab import files

files.download('{}.jpg'.format(save_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>